In [ ]:
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from sqlalchemy import Boolean

from app.infra.sqlalchemy.models import models
from app.infra.sqlalchemy.config.database import get_session, cria_banco
from app.infra.sqlalchemy.repo.repositorios_disciplinas import RepoDisciplina

In [ ]:
cria_banco()

In [ ]:
def horario_valido(horario: str) -> bool:
    if (horario == '') or (horario.startswith('(')) or (horario.startswith('-')) or (horario.endswith(')')):
        return False
    else:
        return True

In [ ]:
def gera_driver() -> webdriver:
    """Função que cria um driver para acessar o navegador"""

    # Define as opções do navegador
    options = webdriver.ChromeOptions()
    options.add_experimental_option("detach", True)

    # Cria o driver
    driver = webdriver.Chrome(options=options)
    # Maximiza a tela
    driver.maximize_window()

    # Define o link a ser acessado
    link = 'https://sigaa.unb.br/sigaa/public/turmas/listar.jsf?aba=p-ensino'

    # Acessa o link
    driver.get(url=link)

    # Retorna o driver
    return driver

In [ ]:
def busca_horarios():
    """Função que busca os horários das disciplinas da UnB"""

    # Chama a função que gera o driver Selenium
    driver = gera_driver()

    # Define o nível de ensino
    nivel_ensino = Select(driver.find_element(By.ID, 'formTurma:inputNivel'))
    nivel_ensino.select_by_value('G')

    # Busca o botão "Ciente" e clica se existir
    btn_ciente = driver.find_element(
        By.XPATH, '//*[@id="sigaa-cookie-consent"]/button')
    if btn_ciente:
        btn_ciente.click()

    # Cria um dicionário vazio
    dicio = {}
    cont_encontradas = 0
    cont_adicionadas = 0

    # Define o departamento
    unidade = Select(driver.find_element(By.ID, 'formTurma:inputDepto'))
    opcoes = unidade.options

    # Percorre todos os departamentos
    # for opcao in opcoes[1::]:
    #     # Resgata o texto da linha do combobox
    #     texto = opcao.text
    try:
        # Seleciona a linha com base no texto
        # unidade.select_by_visible_text(texto)
        unidade.select_by_value('508')

        # Busca o botão "Enviar" e clica se existir
        btn_enviar = driver.find_element(
            By.XPATH, '//*[@id="formTurma"]/table/tfoot/tr/td/input[1]')
        if btn_enviar:
            # Abre em uma nova aba
            btn_enviar.send_keys(Keys.CONTROL + Keys.RETURN)
            driver.switch_to.window(driver.window_handles[1])

        # Busca a tabela com os dados dos horários
        tabela = driver.find_element(
            By.XPATH, '//*[@id="turmasAbertas"]/table')

        # Busca as linhas da tabela
        linhas = tabela.find_elements(By.TAG_NAME, 'tr')

        coluna_qtd = linhas[-1].find_element(By.TAG_NAME, 'td')

        cont_encontradas += int(re.search(r'\d+', coluna_qtd.text).group())

        # Remove a última linha da tabela
        linhas.pop(-1)

        # Percorre as linhas da tabela
        for linha in linhas:
            # Busca os dados da coluna
            colunas = linha.find_elements(By.TAG_NAME, 'td')

            # Armazena a quantidade de colunas
            tamanho_colunas = len(colunas)

            # Se a quantidade for 1, significa que a coluna é o nome da disciplina
            if (tamanho_colunas == 1):
                # Busca o texto da célula
                materia = str(colunas[0].text)

                # Resgata a parte do texto que corresponde ao código da disciplina
                codigo = materia[:7]
                # Resgata a parte do texto que corresponde ao nome da disciplina
                nome = materia[10:].title()

            # Se for 8, significa que as colunas são os dados da disciplina
            elif (tamanho_colunas == 8):
                # depto = texto.title()
                depto = 'CIC'
                turma = str(colunas[0].text)
                periodo = str(colunas[1].text)
                docente = str(colunas[2].text)

                try:
                    match = int(re.search(r'\d+', docente).group())
                    if match:
                        carga_horaria = match

                    else:
                        carga_horaria = 0

                except:
                    print('Não encontrou a carga horária')
                    carga_horaria = 0

                horarios = str(colunas[3].text).split(' ')
                local = str(colunas[7].text)

                id_disciplina = f'{codigo}_{turma}'

                dicio[id_disciplina] = [codigo, nome, False, [horario for horario in horarios if horario_valido(horario)], depto, turma, periodo, docente, carga_horaria, local]

                cont_adicionadas += 1

    except:
        pass

    try:
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    except:
        pass

    print(cont_encontradas)
    print(cont_adicionadas)

    # Fecha o driver Selenium
    driver.close()

    # Retorna o dicionário
    return dicio

In [ ]:
disciplinas = busca_horarios()

In [ ]:
session = next(get_session())

lista_disciplinas = []

for id_disciplina, (codigo, nome, feita, horarios, depto, turma, periodo, docente, carga_horaria, local) in disciplinas.items():
    horarios = '/'.join(horarios)

    disciplina = RepoDisciplina(session).criar(
        id_disciplina=id_disciplina,
        codigo=codigo,
        nome=nome,
        feita=feita,
        depto=depto,
        turma=turma,
        periodo=periodo,
        docente=docente,
        horarios=horarios,
        carga_horaria=carga_horaria,
        local=local
    )
    lista_disciplinas.append(disciplina)

session.add_all(lista_disciplinas)
session.commit()

In [ ]:
def atualizar_disciplina(codigo: str, feita: Boolean):
    """Função que atualiza o parâmetro (feita) de uma disciplina"""

    # Busca a disciplina pelo código
    disciplinas = session.query(models.Disciplina).filter_by(codigo=codigo)

    # Se a disciplina foi encontrada
    if disciplinas:
        # Percorre cada ocorrência da disciplina
        for disciplina in disciplinas:
            # Atualiza o parâmetro
            disciplina.feita = feita

            # Realiza a alteração no banco de dados
            session.commit()

In [ ]:
disciplinas_feitas = [
    # Equivalentes
    'CIC0008',
    'CIC0098',
    'CIC0198',

    # Optativas
    'FTD0007',

    # 1º
    'CIC0003',
    'CIC0004',
    'CIC0247',
    'MAT0025',

    # 2º
    'CIC0002',
    'CIC0090',
    'CIC0229',
    'CIC0231',
    'MAT0026',
    'MAT0031',

    # 3º
    'CIC0142',
    'CIC0197',
    'EST0023',
    'MAT0026',

    # 4º
    'CIC0002',
    'CIC0097',
    'CIC0199',
    'CIC0234',

    # 5º
    'CIC0093',
    'CIC0099',
    'CIC0124',

    # 6º
    'CIC0101',
    'CIC0135',
    'CIC0202',
    'DSC0172',
]

# Percorre a lista de disciplinas a serem atualizadas
for codigo in disciplinas_feitas:
    # Atualiza a disciplina
    atualizar_disciplina(codigo=codigo, feita=True)